## ORM을 사용하는 이유는 객체 지향 언어에서 관계형 데이터베이스를 객체형으로 사용하려.
## ORM이 중간에서 객체형을 관계형으로 전환해주는 역할을 한다.

engine은 데이터베이스 서버와의 연결을 제공해준다.

In [1]:
from sqlalchemy import create_engine

### connecting
- Lazy connecting이다.

In [2]:
engine = create_engine("sqlite:///studyomr1.db")

In [3]:
engine.echo = True

In [4]:
print(engine)

Engine(sqlite:///studyomr1.db)


### Create

#### Table

In [6]:
from sqlalchemy.schema import Table

#### Column

In [7]:
from sqlalchemy.schema import Column

#### MetaData

In [8]:
from sqlalchemy.schema import MetaData

#### 예제

In [10]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey 

### *  MetaData 생성

In [11]:
metadata = MetaData()

## Table 생성
- Table('테이블명', 메타데이타, Columns)
- metadata에 생성

In [12]:
users = Table('users', metadata,
             Column('id', Integer, primary_key=True),
              Column('name', String),
              Column('fullname', String),
             )

In [13]:
addresses = Table('addresses', metadata,
                 Column('id', Integer, primary_key = True),
                  Column('user_id', None, ForeignKey('users.id')),
                  Column('email_address', String, nullable=False)
                 )

## metadata에 생성한 것을 엔진을 통해 데이터베이스에 저장한다.

In [14]:
metadata.create_all(engine)

2019-07-09 19:32:28,889 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 19:32:28,891 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 19:32:28,894 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 19:32:28,895 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 19:32:28,898 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 19:32:28,899 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 19:32:28,902 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 19:32:28,903 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 19:32:28,905 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 19:32:28,907 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 19:32:28,970 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 19:32:28,973 INFO sqlalchemy.engine.b

## Insert

### insert
- insert(values=None, inline=False, \*\*kwargs)

### compile
- compile(bind=None, dialect=None, \*\*kw)
- sql 표현식을 컴파일시킨다.
#### *  리턴값은 컴파일된 객체이다.
- params를 이용하여 컴파일된 객체의 파라미터 이름과 값을 받아올 수 있다.

#### 예제

users table의 insert객체

In [24]:
insert = users.insert()

In [25]:
type(insert)

sqlalchemy.sql.dml.Insert

In [26]:
print(insert)

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)


In [27]:
insert = users.insert().values(name='kim', fullname='Anonymou, Kim ')
print(insert)

INSERT INTO users (name, fullname) VALUES (:name, :fullname)


In [28]:
insert.compile().params

{'name': 'kim', 'fullname': 'Anonymou, Kim '}

# Executing

## Connection

In [30]:
from sqlalchemy.engine import Connection

## execute
- execute(object, \*multiparams, \*\*params)
- SQL문을 생성하고 ResultProxy를 반환한다

## ResultProxy

In [31]:
from sqlalchemy.engine import ResultProxy

### 예제

In [32]:
conn = engine.connect()

In [33]:
conn

### engine과 sql문을 바인딩하여야 데이터베이스로 전송할수 있다.

In [35]:
insert.bind = engine

In [36]:
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

### conn 연결 통로를 통하여 insert를 보내고 그 결과를 result에 담는다.

In [39]:
result = conn.execute(insert)

2019-07-09 19:46:52,059 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 19:46:52,060 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymou, Kim ')
2019-07-09 19:46:52,066 INFO sqlalchemy.engine.base.Engine COMMIT


In [40]:
result.inserted_primary_key

[1]

#### execute의 params를 사용하는 예

In [41]:
insert = users.insert()

In [42]:
result = conn.execute(insert, name="lee", fullname="Unknown, Lee")

2019-07-09 19:49:21,104 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 19:49:21,105 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2019-07-09 19:49:21,110 INFO sqlalchemy.engine.base.Engine COMMIT


#### 결과의 primary_key값을 알려준다.

In [43]:
result.inserted_primary_key

[2]

#### executemany()처럼 사용하고싶다면 다음과 같이 사용하면 된다. multiparams에 list 속 dict형태로 전송하면 된다.

## * addresses에 튜플 추가

In [44]:
conn.execute(addresses.insert(),[
    {'user_id':1, 'email_address':'anonymous.kim@test.com'},
    {'user_id':2, 'email_address':'unknown.lee@test.com'}
])

2019-07-09 19:55:09,314 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 19:55:09,315 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknown.lee@test.com'))
2019-07-09 19:55:09,321 INFO sqlalchemy.engine.base.Engine COMMIT


# select

## select

In [52]:
from sqlalchemy.sql import select

### 예제

In [45]:
query = select([users])

In [46]:
print(query)

SELECT users.id, users.name, users.fullname 
FROM users


In [47]:
result = conn.execute(query)

2019-07-09 19:56:29,408 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 19:56:29,409 INFO sqlalchemy.engine.base.Engine ()


In [48]:
for row in result:
    print(row)

(1, 'kim', 'Anonymou, Kim ')
(2, 'lee', 'Unknown, Lee')


In [49]:
result = conn.execute(select([users.c.name, users.c.fullname]))

2019-07-09 19:57:06,156 INFO sqlalchemy.engine.base.Engine SELECT users.name, users.fullname 
FROM users
2019-07-09 19:57:06,158 INFO sqlalchemy.engine.base.Engine ()


In [51]:
for row in result:
    print(row)

('kim', 'Anonymou, Kim ')
('lee', 'Unknown, Lee')


# ResultProxy

### - fetchone()
    - Fetch one row(한 튜플을 받아온다.)
### - fetchall()
    - Fetch all rows(모든 튜플을 받아온다.)

### 예제 - fetchone()

In [53]:
result = conn.execute(query)

2019-07-09 20:07:07,577 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 20:07:07,579 INFO sqlalchemy.engine.base.Engine ()


In [54]:
row = result.fetchone()

In [55]:
print(row)

(1, 'kim', 'Anonymou, Kim ')


In [57]:
print("id - ", row['id'], ", name - ", row['name'], ", fullname -", row['fullname'])

id -  1 , name -  kim , fullname - Anonymou, Kim 


In [59]:
row = result.fetchone()

In [60]:
print("id - ", row['id'], ", name - ", row['name'], ", fullname -", row['fullname'])

id -  2 , name -  lee , fullname - Unknown, Lee


### 예제- fetchall()

In [61]:
result = conn.execute(query)

2019-07-09 20:09:18,914 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 20:09:18,916 INFO sqlalchemy.engine.base.Engine ()


In [62]:
rows = result.fetchall()

In [63]:
rows

[(1, 'kim', 'Anonymou, Kim '), (2, 'lee', 'Unknown, Lee')]

In [64]:
for row in rows:
    print("id - ", row['id'], ", name - ", row['name'], ", fullname -", row['fullname'])

id -  1 , name -  kim , fullname - Anonymou, Kim 
id -  2 , name -  lee , fullname - Unknown, Lee


In [65]:
result.close()

In [67]:
type(result)

sqlalchemy.engine.result.ResultProxy

### 활용 예제

In [69]:
from sqlalchemy import and_, or_, not_

#### 조건식으로 사용

In [70]:
print(users.c.id == addresses.c.user_id)

users.id = addresses.user_id


In [71]:
print(users.c.id == 1)

users.id = :id_1


In [72]:
print((users.c.id == 1).compile().params)

{'id_1': 1}


In [73]:
print(or_(users.c.id == addresses.c.user_id, users.c.id==1))

users.id = addresses.user_id OR users.id = :id_1


In [74]:
print(and_(users.c.id == addresses.c.user_id, users.c.id == 1))

users.id = addresses.user_id AND users.id = :id_1


In [75]:
print(and_(
        or_(
            users.c.id == addresses.c.user_id,
            users.c.id == 1
        ),
    addresses.c.email_address.like("a%")
    )
)

(users.id = addresses.user_id OR users.id = :id_1) AND addresses.email_address LIKE :email_address_1


In [76]:
print((
    (users.c.id == addresses.c.user_id) |
    (users.c.id == 1)
) & (addresses.c.email_address.like("a%")))

(users.id = addresses.user_id OR users.id = :id_1) AND addresses.email_address LIKE :email_address_1


# selecting

## 예제

In [77]:
result = conn.execute(select([users]).where(users.c.id==1))

2019-07-09 20:16:45,560 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 20:16:45,562 INFO sqlalchemy.engine.base.Engine (1,)


In [78]:
for row in result:
    print(row)

(1, 'kim', 'Anonymou, Kim ')


In [79]:
result = conn.execute(select([users, addresses]).where(users.c.id == addresses.c.user_id))

2019-07-09 20:17:32,364 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 20:17:32,366 INFO sqlalchemy.engine.base.Engine ()


In [80]:
for row in result:
    print(row)

(1, 'kim', 'Anonymou, Kim ', 1, 1, 'anonymous.kim@test.com')
(2, 'lee', 'Unknown, Lee', 2, 2, 'unknown.lee@test.com')


## cross join

In [81]:
result = conn.execute(select([users.c.id, users.c.fullname, addresses.c.email_address])
                     .where(users.c.id==addresses.c.user_id)
                     .where(addresses.c.email_address.like('un%')))

2019-07-09 20:19:13,876 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id AND addresses.email_address LIKE ?
2019-07-09 20:19:13,878 INFO sqlalchemy.engine.base.Engine ('un%',)


In [82]:
for row in result:
    print(row)

(2, 'Unknown, Lee', 'unknown.lee@test.com')


# Join

join(right, onclause=None, isouter=False, full=False)

### 예제

In [83]:
from sqlalchemy import join

### join하면 자동으로 foreignkey를 확인해서 연결해준다

In [84]:
print(users.join(addresses))

users JOIN addresses ON users.id = addresses.user_id


In [85]:
print(users.join(addresses, users.c.id == addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


In [87]:
query = select([users.c.id, users.c.fullname, addresses.c.email_address])\
.select_from(users.join(addresses))

In [89]:
result = conn.execute(query).fetchall()

2019-07-09 20:23:14,959 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 20:23:14,961 INFO sqlalchemy.engine.base.Engine ()


In [90]:
for row in result:
    print(row)

(1, 'Anonymou, Kim ', 'anonymous.kim@test.com')
(2, 'Unknown, Lee', 'unknown.lee@test.com')


In [91]:
artist = Table("Artist", metadata,
              Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              extend_existing=True)

In [92]:
album = Table("Album", metadata,
             Column("id", Integer, primary_key=True),
             Column("title", String, nullable =False),
             Column("artist_id", Integer, ForeignKey("Artist.id")),
             extend_existing=True)

In [99]:
genre = Table("Genre", metadata,
             Column("id", Integer, primary_key = True),
             Column("name", String, nullable=False),
             extend_existing=True)

In [93]:
track = Table("Track", metadata,
             Column("id", Integer, primary_key = True),
             Column("title", String, nullable=False),
             Column("length", Integer, nullable=False),
             Column("rating", Integer, nullable=False),
              Column("count", Integer, nullable=False),
              Column("album_id", Integer, ForeignKey("Album.id")),
              Column("genre_id", Integer, ForeignKey("Genre.id")),
              extend_existing=True)

In [100]:
metadata.create_all(engine)

2019-07-09 20:30:45,862 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 20:30:45,865 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,869 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 20:30:45,870 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,873 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 20:30:45,875 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,877 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 20:30:45,879 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,881 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 20:30:45,883 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,885 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 20:30:45,886 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:30:45,889 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 


In [101]:
tables = metadata.tables

#### metadata에 있는 테이블들

In [102]:
for table in tables:
    print(table)

users
addresses
Artist
Album
Track
Genre


#### db에 있는 테이블들

In [103]:
for table in engine.table_names():
    print(table)

2019-07-09 20:30:54,339 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 20:30:54,341 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track
addresses
users


#### Insert

In [104]:
conn.execute(artist.insert(), [
    {"name":"Led Zepplin"},
    {"name":"AC/DC"}
])

2019-07-09 20:31:41,245 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 20:31:41,247 INFO sqlalchemy.engine.base.Engine (('Led Zepplin',), ('AC/DC',))
2019-07-09 20:31:41,253 INFO sqlalchemy.engine.base.Engine COMMIT


In [105]:
conn.execute(album.insert(),[
    {"title":"IV", "artist_id":1},
    {"title":"Who Made Who", "artist_id":2}
])

2019-07-09 20:32:25,112 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 20:32:25,114 INFO sqlalchemy.engine.base.Engine (('IV', 1), ('Who Made Who', 2))
2019-07-09 20:32:25,118 INFO sqlalchemy.engine.base.Engine COMMIT


In [106]:
conn.execute(genre.insert(),[
    {"name":"Rock"},
    {"name":"Metal"}
])

2019-07-09 20:32:47,655 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 20:32:47,657 INFO sqlalchemy.engine.base.Engine (('Rock',), ('Metal',))
2019-07-09 20:32:47,661 INFO sqlalchemy.engine.base.Engine COMMIT


In [107]:
conn.execute(track.insert(),[
    {"title":"Black Dog", "rating":5, "length":297, "count":0, "album_id":1, "genre_id":1},
    {"title":"Stairway", "rating":5, "length":482, "count":0, "album_id":1, "genre_id":1},
    {"title":"About to rock", "rating":5, "length":313, "count":0, "album_id":2, "genre_id":1},
    {"title":"Who Made Who", "rating":5, "length":297, "count":0, "album_id":2, "genre_id":1},
])

2019-07-09 20:35:00,716 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 20:35:00,717 INFO sqlalchemy.engine.base.Engine (('Black Dog', 297, 5, 0, 1, 1), ('Stairway', 482, 5, 0, 1, 1), ('About to rock', 313, 5, 0, 2, 1), ('Who Made Who', 297, 5, 0, 2, 1))
2019-07-09 20:35:00,737 INFO sqlalchemy.engine.base.Engine COMMIT


### select

In [108]:
artistResult = conn.execute(artist.select())
for row in artistResult:
    print(row)

2019-07-09 20:35:48,748 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id, "Artist".name 
FROM "Artist"
2019-07-09 20:35:48,750 INFO sqlalchemy.engine.base.Engine ()
(1, 'Led Zepplin')
(2, 'AC/DC')


In [109]:
albumResult = conn.execute(album.select())
for row in albumResult:
    print(row)

2019-07-09 20:36:07,838 INFO sqlalchemy.engine.base.Engine SELECT "Album".id, "Album".title, "Album".artist_id 
FROM "Album"
2019-07-09 20:36:07,840 INFO sqlalchemy.engine.base.Engine ()
(1, 'IV', 1)
(2, 'Who Made Who', 2)


In [110]:
genreResult = conn.execute(genre.select())
for row in genreResult:
    print(row)

2019-07-09 20:36:28,591 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id, "Genre".name 
FROM "Genre"
2019-07-09 20:36:28,593 INFO sqlalchemy.engine.base.Engine ()
(1, 'Rock')
(2, 'Metal')


In [112]:
trackResult = conn.execute(track.select())
for row in trackResult:
    print(row)

2019-07-09 20:36:56,933 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track"
2019-07-09 20:36:56,935 INFO sqlalchemy.engine.base.Engine ()
(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)
(3, 'About to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 1)


### Where

In [113]:
trackResult = conn.execute(select([track])
                           .where(
                           and_(track.c.album_id==1, track.c.genre_id == 1)))

2019-07-09 20:38:03,698 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND "Track".genre_id = ?
2019-07-09 20:38:03,699 INFO sqlalchemy.engine.base.Engine (1, 1)


In [114]:
for row in trackResult:
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 1)


### Update

In [115]:
from sqlalchemy import update

In [116]:
conn.execute(track.update().values(genre_id = 2).where(track.c.id==2))

2019-07-09 20:39:29,780 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2019-07-09 20:39:29,781 INFO sqlalchemy.engine.base.Engine (2, 2)
2019-07-09 20:39:29,786 INFO sqlalchemy.engine.base.Engine COMMIT


In [117]:
conn.execute(track.update().values(genre_id = 1).where(track.c.id==3))

2019-07-09 20:40:32,659 INFO sqlalchemy.engine.base.Engine UPDATE "Track" SET genre_id=? WHERE "Track".id = ?
2019-07-09 20:40:32,661 INFO sqlalchemy.engine.base.Engine (1, 3)
2019-07-09 20:40:32,664 INFO sqlalchemy.engine.base.Engine COMMIT


### Where

In [118]:
trackResult = conn.execute(select([track])
                          .where(
                              and_(track.c.album_id == 1,
                                  or_(track.c.genre_id ==1,
                                     track.c.genre_id ==2))
                          ))

2019-07-09 20:41:53,062 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" 
WHERE "Track".album_id = ? AND ("Track".genre_id = ? OR "Track".genre_id = ?)
2019-07-09 20:41:53,063 INFO sqlalchemy.engine.base.Engine (1, 1, 2)


In [119]:
for row in trackResult:
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)


### Join

In [120]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [124]:
result = conn.execute(track.select().select_from(track.join(album)))

2019-07-09 20:43:09,764 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id
2019-07-09 20:43:09,766 INFO sqlalchemy.engine.base.Engine ()


In [125]:
for row in result.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)
(3, 'About to rock', 313, 5, 0, 2, 1)
(4, 'Who Made Who', 297, 5, 0, 2, 1)


In [126]:
result = conn.execute(track
                     .select()
                     .select_from(track.join(album))
                     .where(album.c.id==1))

2019-07-09 20:43:50,436 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id 
WHERE "Album".id = ?
2019-07-09 20:43:50,438 INFO sqlalchemy.engine.base.Engine (1,)


In [127]:
for row in result.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)
(2, 'Stairway', 482, 5, 0, 1, 2)


# Multiple Join

In [128]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [129]:
print(track.join(album).join(genre))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id


In [131]:
print(track.join(album).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [132]:
print(track.join(album).join(genre).join(artist))

"Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id


In [133]:
result = conn.execute(select([track.c.title, album.c.title, genre.c.name, artist.c.name])
                     .select_from(track.join(album).join(genre).join(artist)))

2019-07-09 20:46:43,257 INFO sqlalchemy.engine.base.Engine SELECT "Track".title, "Album".title, "Genre".name, "Artist".name 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 20:46:43,260 INFO sqlalchemy.engine.base.Engine ()


In [134]:
for row in result.fetchall():
    print(row)

('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Metal', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Rock', 'AC/DC')


In [135]:
result = conn.execute(track
                     .select()
                     .select_from(track.join(album).join(genre).join(artist))
                     .where(
                         and_(genre.c.id==1,
                             artist.c.id==1,
                             )
                     ))

2019-07-09 20:48:03,493 INFO sqlalchemy.engine.base.Engine SELECT "Track".id, "Track".title, "Track".length, "Track".rating, "Track".count, "Track".album_id, "Track".genre_id 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id 
WHERE "Genre".id = ? AND "Artist".id = ?
2019-07-09 20:48:03,495 INFO sqlalchemy.engine.base.Engine (1, 1)


In [136]:
for row in result.fetchall():
    print(row)

(1, 'Black Dog', 297, 5, 0, 1, 1)


### 연결 종료와 메타데이터 클리어

In [137]:
conn.close()
metadata.clear()

## Declare

## declarative_base

In [138]:
from sqlalchemy.ext.declarative import declarative_base

In [139]:
from sqlalchemy import create_engine

In [140]:
engine = create_engine("sqlite:///sqlomr2.db", echo=True)

## 매핑하고 싶은 클래스에 상속시킨다.

In [141]:
base = declarative_base()

In [142]:
class User(base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname= Column(String)
    password = Column("passwd", String)
    
    def __repr__(self):
        return "<T'User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

In [143]:
User.__table__

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('passwd', String(), table=<users>), schema=None)

In [144]:
base.metadata.create_all(engine)

2019-07-09 20:57:23,497 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 20:57:23,500 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:57:23,502 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 20:57:23,504 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:57:23,506 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 20:57:23,507 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:57:23,510 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	passwd VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 20:57:23,511 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 20:57:23,590 INFO sqlalchemy.engine.base.Engine COMMIT


In [145]:
kim = User(name = "kim", fullname = "anonymous, Kim", password="kimbap nara")

In [146]:
print(kim)

<T'User(name='kim', fullname='anonymous, Kim', password='kimbap nara')>


In [147]:
print(kim.id)

None


# Session

## Session
- 세션은 데이터베이스와의 모든 대화를 설정하고 모든 개체를 나타냅니다.
- ORM으로 매핑 된 객체에 대한 지속성 작업을 관리합니다.

In [148]:
from sqlalchemy.orm.session import sessionmaker

In [149]:
Session = sessionmaker(bind=engine)

In [150]:
session = Session()

# Insert

## add

### Pending된다. commit까지 기다림

In [151]:
session.add(kim)

In [152]:
session.add_all([
    User(name="lee", fullname="unknown, Lee", password="hello"),
    User(name="park", fullname="nobody, Park", password = "pass Park")
])

# Update

## dirty
- 삭제가 아닌 수정을 할 때 사용한다.

In [158]:
kim.password = "all"

session.dirty

IdentitySet([<T'User(name='kim', fullname='anonymous, Kim', password='all')>])

In [159]:
session.is_modified(kim)

True

In [160]:
session.commit()

2019-07-09 21:07:10,797 INFO sqlalchemy.engine.base.Engine UPDATE users SET passwd=? WHERE users.id = ?
2019-07-09 21:07:10,799 INFO sqlalchemy.engine.base.Engine ('all', 1)
2019-07-09 21:07:10,808 INFO sqlalchemy.engine.base.Engine COMMIT


# Select
- query

In [161]:
from sqlalchemy.orm import Query

In [162]:
for row in session.query(User):
    print(type(row))
    print(row.id, row.name, row.fullname, row.password)

2019-07-09 21:08:31,075 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:08:31,078 INFO sqlalchemy.engine.base.Engine SELECT users.passwd AS users_passwd, users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users
2019-07-09 21:08:31,079 INFO sqlalchemy.engine.base.Engine ()
<class '__main__.User'>
1 kim anonymous, Kim all
<class '__main__.User'>
2 lee unknown, Lee hello
<class '__main__.User'>
3 park nobody, Park pass Park


In [164]:
str(session.query(User))

'SELECT users.passwd AS users_passwd, users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname \nFROM users'

### filter
- 필터링에 사용
- 파이썬 연산자로 계산한다.

### filter_by
- keyword 표현식으로 사용

In [165]:
for row in session.query(User.id, User.fullname).filter(User.name == "lee"):
    print(type(row))
    print(row.id, row.fullname)

2019-07-09 21:10:46,263 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2019-07-09 21:10:46,264 INFO sqlalchemy.engine.base.Engine ('lee',)
<class 'sqlalchemy.util._collections.result'>
2 unknown, Lee


In [167]:
for row in session.query(User.id, User.fullname).filter_by(name = 'lee'):
    print(type(row))
    print(row.id, row.fullname)

2019-07-09 21:11:52,143 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.fullname AS users_fullname 
FROM users 
WHERE users.name = ?
2019-07-09 21:11:52,146 INFO sqlalchemy.engine.base.Engine ('lee',)
<class 'sqlalchemy.util._collections.result'>
2 unknown, Lee


In [168]:
class Artist(base):
    __tablename__ = "Artist"
    
    id = Column("id", Integer, primary_key=True)
    name = Column("name", String, nullable=False)

In [169]:
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    

In [170]:
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)

In [171]:
class Track(base):
    __tablename__="Track"
    
    id = Column("id", Integer, primary_key = True)
    title = Column("title", String, nullable=False)
    length = Column("length", Integer, nullable=False)
    rating = Column("rating", Integer, nullable=False)
    count = Column("count", Integer, nullable=False)
    album_id = Column("album_id", Integer, ForeignKey("Album.id"))
    genre_id = Column("genre_id", Integer, ForeignKey("Genre.id"))

In [179]:
base.metadata.create_all(engine)

2019-07-09 21:19:26,336 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 21:19:26,338 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,341 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 21:19:26,344 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,346 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 21:19:26,347 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,348 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 21:19:26,349 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,351 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 21:19:26,352 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,354 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2019-07-09 21:19:26,355 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:19:26,409 INFO sqlalchemy.eng

In [183]:
artist1 = Artist(name="Led Zepplin")
artist2 = Artist(name = "AC/DC")

In [184]:
session.add_all([artist1, artist2])

In [185]:
session.commit()

2019-07-09 21:19:45,583 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:19:45,585 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 21:19:45,588 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2019-07-09 21:19:45,596 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 21:19:45,598 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2019-07-09 21:19:45,601 INFO sqlalchemy.engine.base.Engine COMMIT


In [186]:
album = [Album(title = "IV", artist_id = artist1.id),
        Album(title="Who Made Who", artist_id=artist2.id)]

2019-07-09 21:20:46,514 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:20:46,517 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 21:20:46,518 INFO sqlalchemy.engine.base.Engine (1,)
2019-07-09 21:20:46,523 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 21:20:46,524 INFO sqlalchemy.engine.base.Engine (2,)


In [187]:
session.add_all(album)

In [188]:
session.commit()

2019-07-09 21:20:56,328 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 21:20:56,330 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2019-07-09 21:20:56,334 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 21:20:56,336 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 2)
2019-07-09 21:20:56,339 INFO sqlalchemy.engine.base.Engine COMMIT


In [189]:
session.add_all([
    Genre(name="Rock"), Genre(name="Metal")
])

In [190]:
session.commit()

2019-07-09 21:21:27,014 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:21:27,017 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 21:21:27,019 INFO sqlalchemy.engine.base.Engine ('Rock',)
2019-07-09 21:21:27,034 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 21:21:27,035 INFO sqlalchemy.engine.base.Engine ('Metal',)
2019-07-09 21:21:27,037 INFO sqlalchemy.engine.base.Engine COMMIT


In [191]:
album1 = session.query(Album).filter(Album.artist_id == artist1.id).one()

2019-07-09 21:21:55,544 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:21:55,547 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 21:21:55,549 INFO sqlalchemy.engine.base.Engine (1,)
2019-07-09 21:21:55,554 INFO sqlalchemy.engine.base.Engine SELECT "Album".id AS "Album_id", "Album".title AS "Album_title", "Album".artist_id AS "Album_artist_id" 
FROM "Album" 
WHERE "Album".artist_id = ?
2019-07-09 21:21:55,556 INFO sqlalchemy.engine.base.Engine (1,)


In [192]:
album2 = session.query(Album).filter(Album.artist_id==artist2.id).one()

2019-07-09 21:22:21,633 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist" 
WHERE "Artist".id = ?
2019-07-09 21:22:21,635 INFO sqlalchemy.engine.base.Engine (2,)
2019-07-09 21:22:21,639 INFO sqlalchemy.engine.base.Engine SELECT "Album".id AS "Album_id", "Album".title AS "Album_title", "Album".artist_id AS "Album_artist_id" 
FROM "Album" 
WHERE "Album".artist_id = ?
2019-07-09 21:22:21,640 INFO sqlalchemy.engine.base.Engine (2,)


In [193]:
genre1 = session.query(Genre).filter(Genre.name=="Rock").filter(Genre.id==1).one()

2019-07-09 21:23:07,814 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id AS "Genre_id", "Genre".name AS "Genre_name" 
FROM "Genre" 
WHERE "Genre".name = ? AND "Genre".id = ?
2019-07-09 21:23:07,815 INFO sqlalchemy.engine.base.Engine ('Rock', 1)


In [196]:
genre2 = session.query(Genre).filter(Genre.id==2).one()

2019-07-09 21:27:07,438 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id AS "Genre_id", "Genre".name AS "Genre_name" 
FROM "Genre" 
WHERE "Genre".id = ?
2019-07-09 21:27:07,440 INFO sqlalchemy.engine.base.Engine (2,)


In [197]:
track = [Track(title = "Black Dog", rating=5, length = 297, count = 0, album_id = album1.id, genre_id = genre1.id),
        Track(title = "Stairway", rating=5, length = 482, count = 0, album_id = album1.id, genre_id = genre2.id),
        Track(title = "About to rock", rating=5, length = 313, count = 0, album_id = album2.id, genre_id = genre1.id),
        Track(title = "Who Made Who", rating=5, length = 297, count = 0, album_id = album2.id, genre_id = genre2.id),
        ]

In [198]:
session.add_all(track)

In [199]:
session.commit()

2019-07-09 21:27:22,997 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 21:27:22,999 INFO sqlalchemy.engine.base.Engine ('Black Dog', 297, 5, 0, 1, 1)
2019-07-09 21:27:23,003 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 21:27:23,006 INFO sqlalchemy.engine.base.Engine ('Stairway', 482, 5, 0, 1, 2)
2019-07-09 21:27:23,008 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 21:27:23,010 INFO sqlalchemy.engine.base.Engine ('About to rock', 313, 5, 0, 2, 1)
2019-07-09 21:27:23,013 INFO sqlalchemy.engine.base.Engine INSERT INTO "Track" (title, length, rating, count, album_id, genre_id) VALUES (?, ?, ?, ?, ?, ?)
2019-07-09 21:27:23,015 INFO sqlalchemy.engine.base.Engine ('Who Made Who', 297, 5, 0, 2, 2)
2019-07-09

## Join

In [200]:
result = session.query(Track.title, Album.title, Genre.name, Artist.name)\
.select_from(Track).join(Album).join(Genre).join(Artist).all()

2019-07-09 21:28:33,265 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:28:33,268 INFO sqlalchemy.engine.base.Engine SELECT "Track".title AS "Track_title", "Album".title AS "Album_title", "Genre".name AS "Genre_name", "Artist".name AS "Artist_name" 
FROM "Track" JOIN "Album" ON "Album".id = "Track".album_id JOIN "Genre" ON "Genre".id = "Track".genre_id JOIN "Artist" ON "Artist".id = "Album".artist_id
2019-07-09 21:28:33,270 INFO sqlalchemy.engine.base.Engine ()


In [201]:
for row in result:
    print(row)

('Black Dog', 'IV', 'Rock', 'Led Zepplin')
('Stairway', 'IV', 'Metal', 'Led Zepplin')
('About to rock', 'Who Made Who', 'Rock', 'AC/DC')
('Who Made Who', 'Who Made Who', 'Metal', 'AC/DC')


In [204]:
songList = [dict(Track = row[0], Album=row[1], Genre=row[2], Artist=row[3]) for row in result]

In [205]:
songList

[{'Track': 'Black Dog',
  'Album': 'IV',
  'Genre': 'Rock',
  'Artist': 'Led Zepplin'},
 {'Track': 'Stairway',
  'Album': 'IV',
  'Genre': 'Metal',
  'Artist': 'Led Zepplin'},
 {'Track': 'About to rock',
  'Album': 'Who Made Who',
  'Genre': 'Rock',
  'Artist': 'AC/DC'},
 {'Track': 'Who Made Who',
  'Album': 'Who Made Who',
  'Genre': 'Metal',
  'Artist': 'AC/DC'}]

### Relationship
- 두 개의 매핑된 클래스 사이에 관계를 제공한다.
- 부모-자식 또는 연관된 테이블 관계를 대응시켜준다.

In [206]:
metadata = MetaData()

In [209]:
engine = create_engine("sqlite:///sqlomr3.db", echo = True)

In [210]:
Session = sessionmaker(engine)

In [211]:
session = Session()

In [214]:
class User(base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    fullname = Column(String)
    password=Column("passwd", String)

InvalidRequestError: Table 'users' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [218]:
class Address(base):
    __tablename__="addresses"
    
    id = Column(Integer, primary_key = True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))

In [220]:
base.metadata.create_all(engine)

2019-07-09 21:49:15,057 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 21:49:15,059 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:49:15,062 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 21:49:15,064 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:49:15,066 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 21:49:15,068 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:49:15,070 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 21:49:15,072 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:49:15,075 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 21:49:15,077 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:49:15,151 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 21:49:15,154 INFO sqlalchemy.engine.base.Engine 
CREATE T

In [222]:
from sqlalchemy.orm import relationship

In [223]:
class Artist(base):
    __tablename__ = "Artist"
    
    id = Column("id", Integer, primary_key=True)
    name = Column("name", String, nullable=False)
    
    albumList = relationship("Album", back_populates="artist")
    
class Album(base):
    __tablename__ = "Album"
    
    id = Column(Integer, primary_key = True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("Artist.id"))
    
    artist = relationship("Artist", back_populates="albumList", uselist=False)
    trackList = relationship("Track", back_populates="album")
    
class Genre(base):
    __tablename__ = "Genre"
    
    id = Column(Integer, primary_key = True)
    name = Column(String)
    
    trackList = relationship("Track", back_populates="genre")
    
class Track(base):
    __tablename__="Track"
    
    id = Column("id", Integer, primary_key = True)
    title = Column("title", String, nullable=False)
    length = Column("length", Integer, nullable=False)
    rating = Column("rating", Integer, nullable=False)
    count = Column("count", Integer, nullable=False)
    album_id = Column("album_id", Integer, ForeignKey("Album.id"))
    genre_id = Column("genre_id", Integer, ForeignKey("Genre.id"))
    
    album = relationship("Album", back_populates = "trackList", uselist=False)
    genre = relationship("Genre", back_populates="trackList", uselist = False)

In [224]:
base.metadata.create_all(engine)

2019-07-09 21:54:41,354 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 21:54:41,356 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,361 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 21:54:41,363 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,365 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 21:54:41,367 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,368 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 21:54:41,370 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,371 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 21:54:41,372 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,374 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 21:54:41,375 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 21:54:41,378 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Artist" (
	id INTEGER NOT NULL, 


In [227]:
track1 = Track(title = "Black Dog", rating=5, length = 297, count = 0)
track2 = Track(title = "Stairway", rating=5, length = 482, count = 0)
track3 = Track(title = "About to rock", rating=5, length = 313, count = 0)
track4 = Track(title = "Who Made Who", rating=5, length = 297, count = 0)  

In [228]:
track1.album = track2.album = Album(title = "IV")

In [229]:
track3.album = track4.album = Album(title="Who Made Who")

In [230]:
track1.genre = track3.genre = Genre(name="Rock")
track2.genre = track4.genre = Genre(name="Metal")

In [231]:
track1.album.artist = track2.album.artist = Artist(name="Led Zepplin")
track3.album.artist = track4.album.artist = Artist(name ="AC/DC")

In [232]:
session.add_all([track1, track2, track3, track4])

In [233]:
session.commit()

2019-07-09 21:59:10,507 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-09 21:59:10,510 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 21:59:10,512 INFO sqlalchemy.engine.base.Engine ('Metal',)
2019-07-09 21:59:10,517 INFO sqlalchemy.engine.base.Engine INSERT INTO "Genre" (name) VALUES (?)
2019-07-09 21:59:10,519 INFO sqlalchemy.engine.base.Engine ('Rock',)
2019-07-09 21:59:10,521 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 21:59:10,523 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2019-07-09 21:59:10,524 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 21:59:10,526 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2019-07-09 21:59:10,528 INFO sqlalchemy.engine.base.Engine INSERT INTO "Album" (title, artist_id) VALUES (?, ?)
2019-07-09 21:59:10,530 INFO sqlalchemy.engine.base.Engine ('IV', 1)
2019-07-09 21:59:10,533 INFO sqlalchemy.engine.base.Engine INSERT IN

In [235]:
result = session.query(Artist)

In [238]:
for row in result:
    print(row.name)

2019-07-09 22:00:52,544 INFO sqlalchemy.engine.base.Engine SELECT "Artist".id AS "Artist_id", "Artist".name AS "Artist_name" 
FROM "Artist"
2019-07-09 22:00:52,546 INFO sqlalchemy.engine.base.Engine ()
Led Zepplin
AC/DC
